In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import sys
import glob
import os
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from sklearn.model_selection import ParameterGrid
from pathlib import Path
import pickle

game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]

In [29]:
def get_sats_games_from_prefixes(prefixes):
    game_list = []
    for prefix in prefixes:
        game_list += [f'{prefix}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{prefix}/*')]
    return sorted(set(game_list))

# can apply other filters like:
# sats_games = [s for s in sats_games if ('hide' not in s) and '1000' in s and 'risk_averse' not in s]
# sats_games = sats_games[:1] # debug

In [15]:
# feb10_2p_v1: 2-player games with 1-7 types
sats_games = get_sats_games_from_prefixes([f'feb10_2p_{t}type' for t in [1, 2, 3, 5, 7]])
BASE_JOB_NAME = 'feb10_2p_v1'
N_SEEDS = 10 # Number of seeds of each experiment to launch
time_limit_seconds = int(3600 * 1)
num_iters = 999_000_000
extra_name = ''
overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 1000 --time_limit_seconds {time_limit_seconds} --use_wandb true'
pgrid = ParameterGrid([
    {'game': [g for g in sats_games if 'rho1' in g][:], 'alg_and_config': [('cfr', 'cfr_10')]}, # only against trembling opponents
    {'game': [g for g in sats_games if 'rho0' in g][:], 'alg_and_config': [('cfr', 'cfr_10_no_trem')]}, # only against regular opponents
])
 
# ----------------------------------------------------

print(f'Games: {len(sats_games)}; Grid: {len(pgrid)}')
for params in pgrid:
    game_name = params['game']
    (alg_name, alg_config) = params['alg_and_config']
    dispatch.dispatch_experiments(alg_config, base_job_name=BASE_JOB_NAME, game_name=game_name, overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg=alg_name, cluster='ada_vickrey')

Games: 100; Grid: 100
Submitted batch job 1085236
Submitted batch job 1085237
Submitted batch job 1085238
Submitted batch job 1085239
Submitted batch job 1085240
Submitted batch job 1085241
Submitted batch job 1085242
Submitted batch job 1085243
Submitted batch job 1085244
Submitted batch job 1085245
Submitted batch job 1085246
Submitted batch job 1085247
Submitted batch job 1085248
Submitted batch job 1085249
Submitted batch job 1085250
Submitted batch job 1085251
Submitted batch job 1085252
Submitted batch job 1085253
Submitted batch job 1085254
Submitted batch job 1085255
Submitted batch job 1085256
Submitted batch job 1085257
Submitted batch job 1085258
Submitted batch job 1085259
Submitted batch job 1085260
Submitted batch job 1085261
Submitted batch job 1085262
Submitted batch job 1085263
Submitted batch job 1085264
Submitted batch job 1085265
Submitted batch job 1085266
Submitted batch job 1085267
Submitted batch job 1085268
Submitted batch job 1085269
Submitted batch job 108527

In [13]:
# feb10_speed_v1: 5 type games with frequent NashConv
sats_games = get_sats_games_from_prefixes(['feb10_2p_5type'])
BASE_JOB_NAME = 'feb10_speed_v1'
N_SEEDS = 10 # Number of seeds of each experiment to launch
time_limit_seconds = int(3600 * 1)
num_iters = 999_000_000
extra_name = ''
overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 100 --compute_nash_conv true --time_limit_seconds {time_limit_seconds} --use_wandb true'
pgrid = ParameterGrid([
    {'game': [g for g in sats_games if 'rho1' in g][:], 'alg_and_config': [('cfr', 'cfr_10')]}, # only against trembling opponents
    {'game': [g for g in sats_games if 'rho0' in g][:], 'alg_and_config': [('cfr', 'cfr_10_no_trem')]}, # only against regular opponents
])
 
# ----------------------------------------------------

print(f'Games: {len(sats_games)}; Grid: {len(pgrid)}')
for params in pgrid:
    game_name = params['game']
    (alg_name, alg_config) = params['alg_and_config']
    dispatch.dispatch_experiments(alg_config, base_job_name=BASE_JOB_NAME, game_name=game_name, overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg=alg_name, cluster='ada')

Games: 20; Grid: 20
Submitted batch job 1085036
Submitted batch job 1085037
Submitted batch job 1085038
Submitted batch job 1085039
Submitted batch job 1085040
Submitted batch job 1085041
Submitted batch job 1085042
Submitted batch job 1085043
Submitted batch job 1085044
Submitted batch job 1085045
Submitted batch job 1085046
Submitted batch job 1085047
Submitted batch job 1085048
Submitted batch job 1085049
Submitted batch job 1085050
Submitted batch job 1085051
Submitted batch job 1085052
Submitted batch job 1085053
Submitted batch job 1085054
Submitted batch job 1085055
Submitted batch job 1085056
Submitted batch job 1085057
Submitted batch job 1085058
Submitted batch job 1085059
Submitted batch job 1085060
Submitted batch job 1085061
Submitted batch job 1085062
Submitted batch job 1085063
Submitted batch job 1085064
Submitted batch job 1085065
Submitted batch job 1085066
Submitted batch job 1085067
Submitted batch job 1085068
Submitted batch job 1085069
Submitted batch job 1085070


In [12]:
# feb8_3p_5supply_v2: 3 players, 5 encumbered licenses. Probably too big to run, but let's try...
sats_games = get_sats_games_from_prefixes(['feb8_3p_5supply'])
BASE_JOB_NAME = 'feb8_3p_5supply_v2'
N_SEEDS = 5 # Number of seeds of each experiment to launch
time_limit_seconds = int(3600 * 8)
num_iters = 999_000_000
extra_name = ''
overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 1000 --time_limit_seconds {time_limit_seconds} --use_wandb true'
pgrid = ParameterGrid([
    {'game': [g for g in sats_games if 'rho1' in g][:], 'alg_and_config': [('cfr', 'cfr_10')]}, # only against trembling opponents
    {'game': [g for g in sats_games if 'rho0' in g][:], 'alg_and_config': [('cfr', 'cfr_10_no_trem')]}, # only against regular opponents
])
 
# ----------------------------------------------------

print(f'Games: {len(sats_games)}; Grid: {len(pgrid)}')
for params in pgrid:
    game_name = params['game']
    (alg_name, alg_config) = params['alg_and_config']
    dispatch.dispatch_experiments(alg_config, base_job_name=BASE_JOB_NAME, game_name=game_name, overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg=alg_name)

Games: 20; Grid: 20
Submitted batch job 1084926
Submitted batch job 1084927
Submitted batch job 1084928
Submitted batch job 1084929
Submitted batch job 1084930
Submitted batch job 1084931
Submitted batch job 1084932
Submitted batch job 1084933
Submitted batch job 1084934
Submitted batch job 1084935
Submitted batch job 1084936
Submitted batch job 1084937
Submitted batch job 1084938
Submitted batch job 1084939
Submitted batch job 1084940
Submitted batch job 1084941
Submitted batch job 1084942
Submitted batch job 1084943
Submitted batch job 1084944
Submitted batch job 1084945
Submitted batch job 1084946
Submitted batch job 1084947
Submitted batch job 1084948
Submitted batch job 1084949
Submitted batch job 1084950
Submitted batch job 1084951
Submitted batch job 1084952
Submitted batch job 1084953
Submitted batch job 1084954
Submitted batch job 1084955
Submitted batch job 1084956
Submitted batch job 1084957
Submitted batch job 1084958
Submitted batch job 1084959
Submitted batch job 1084960


In [10]:
# feb8_3p_5supply_v1: 3 players, 5 encumbered licenses. Probably too big to run, but let's try...
sats_games = get_sats_games_from_prefixes(['feb8_3p_5supply'])
BASE_JOB_NAME = 'feb8_3p_5supply_v1'
N_SEEDS = 5 # Number of seeds of each experiment to launch
time_limit_seconds = int(3600 * 3)
num_iters = 999_000_000
extra_name = ''
overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 1000 --time_limit_seconds {time_limit_seconds} --use_wandb true'
pgrid = ParameterGrid([{
    'game': [g for g in sats_games][:],
    'alg_and_config': [
        ('cfr', 'cfr_10'), # only against trembling opponents
        ('ppo', 'ppo_jun8_23')
    ]
}])
 
# ----------------------------------------------------

print(f'Games: {len(sats_games)}; Grid: {len(pgrid)}')
for params in pgrid:
    game_name = params['game']
    (alg_name, alg_config) = params['alg_and_config']
    dispatch.dispatch_experiments(alg_config, base_job_name=BASE_JOB_NAME, game_name=game_name, overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg=alg_name)

Games: 20; Grid: 40
Submitted batch job 1084723
Submitted batch job 1084724
Submitted batch job 1084725
Submitted batch job 1084726
Submitted batch job 1084727
Submitted batch job 1084728
Submitted batch job 1084729
Submitted batch job 1084730
Submitted batch job 1084731
Submitted batch job 1084732
Submitted batch job 1084733
Submitted batch job 1084734
Submitted batch job 1084735
Submitted batch job 1084736
Submitted batch job 1084737
Submitted batch job 1084738
Submitted batch job 1084739
Submitted batch job 1084740
Submitted batch job 1084741
Submitted batch job 1084742
Submitted batch job 1084743
Submitted batch job 1084744
Submitted batch job 1084745
Submitted batch job 1084746
Submitted batch job 1084747
Submitted batch job 1084748
Submitted batch job 1084749
Submitted batch job 1084750
Submitted batch job 1084751
Submitted batch job 1084752
Submitted batch job 1084753
Submitted batch job 1084754
Submitted batch job 1084755
Submitted batch job 1084756
Submitted batch job 1084757


In [9]:
# feb8_3p_7supply_v2: 3 players, 7 encumbered licenses
sats_games = get_sats_games_from_prefixes(['feb8_3p_7supply'])
BASE_JOB_NAME = 'feb8_3p_7supply_v2'
N_SEEDS = 5 # Number of seeds of each experiment to launch
time_limit_seconds = int(3600 * 3)
num_iters = 999_000_000
extra_name = ''
overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 1000 --time_limit_seconds {time_limit_seconds} --use_wandb true'
pgrid = ParameterGrid([{
    'game': [g for g in sats_games][:],
    'alg_and_config': [
        ('cfr', 'cfr_10'), # only against trembling opponents
        ('ppo', 'ppo_jun8_23')
    ]
}])
 
# ----------------------------------------------------

print(f'Games: {len(sats_games)}; Grid: {len(pgrid)}')
for params in pgrid:
    game_name = params['game']
    (alg_name, alg_config) = params['alg_and_config']
    dispatch.dispatch_experiments(alg_config, base_job_name=BASE_JOB_NAME, game_name=game_name, overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg=alg_name)

Games: 20; Grid: 40
Submitted batch job 1084523
Submitted batch job 1084524
Submitted batch job 1084525
Submitted batch job 1084526
Submitted batch job 1084527
Submitted batch job 1084528
Submitted batch job 1084529
Submitted batch job 1084530
Submitted batch job 1084531
Submitted batch job 1084532
Submitted batch job 1084533
Submitted batch job 1084534
Submitted batch job 1084535
Submitted batch job 1084536
Submitted batch job 1084537
Submitted batch job 1084538
Submitted batch job 1084539
Submitted batch job 1084540
Submitted batch job 1084541
Submitted batch job 1084542
Submitted batch job 1084543
Submitted batch job 1084544
Submitted batch job 1084545
Submitted batch job 1084546
Submitted batch job 1084547
Submitted batch job 1084548
Submitted batch job 1084549
Submitted batch job 1084550
Submitted batch job 1084551
Submitted batch job 1084552
Submitted batch job 1084553
Submitted batch job 1084554
Submitted batch job 1084555
Submitted batch job 1084556
Submitted batch job 1084557


In [6]:
# feb8_v1: re-run of feb5_v2 with PPO fixed and no intermediate heuristicconv
sats_games = get_sats_games_from_prefixes(['feb4_4t'])
BASE_JOB_NAME = 'feb8_v1'
N_SEEDS = 11 # Number of seeds of each experiment to launch
time_limit_seconds = int(3600 * 1)
num_iters = 999_000_000
extra_name = ''
overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 1000 --time_limit_seconds {time_limit_seconds} --use_wandb true'
pgrid = ParameterGrid([{
    'game': [g for g in sats_games][:],
    'alg_and_config': [
        ('cfr', 'cfr_port_10_ext'), 
        ('ppo', 'ppo_jun8_23')
    ]
}])
 
# ----------------------------------------------------

print(f'Games: {len(sats_games)}; Grid: {len(pgrid)}')
for params in pgrid:
    game_name = params['game']
    (alg_name, alg_config) = params['alg_and_config']
    dispatch.dispatch_experiments(alg_config, base_job_name=BASE_JOB_NAME, game_name=game_name, overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg=alg_name)

Games: 20; Grid: 40
Submitted batch job 1083412
Submitted batch job 1083413
Submitted batch job 1083414
Submitted batch job 1083415
Submitted batch job 1083416
Submitted batch job 1083417
Submitted batch job 1083418
Submitted batch job 1083419
Submitted batch job 1083420
Submitted batch job 1083421
Submitted batch job 1083422
Submitted batch job 1083423
Submitted batch job 1083424
Submitted batch job 1083425
Submitted batch job 1083426
Submitted batch job 1083427
Submitted batch job 1083428
Submitted batch job 1083429
Submitted batch job 1083430
Submitted batch job 1083431
Submitted batch job 1083432
Submitted batch job 1083433
Submitted batch job 1083434
Submitted batch job 1083435
Submitted batch job 1083436
Submitted batch job 1083437
Submitted batch job 1083438
Submitted batch job 1083439
Submitted batch job 1083440
Submitted batch job 1083441
Submitted batch job 1083442
Submitted batch job 1083443
Submitted batch job 1083444
Submitted batch job 1083445
Submitted batch job 1083446


In [ ]:
# feb7_v1: re-run of feb6_v1, but with torch deterministic and no intermediate heuristicconv
sats_games = get_sats_games_from_prefixes(['feb4_4t'])
pgrid = ParameterGrid([{
    'game': [g for g in sats_games][:],
    'alg_and_config': [('ppo', 'ppo_jun8_23')]
}])
BASE_JOB_NAME = 'feb7_v1'
N_SEEDS = 20 # Number of seeds of each experiment to launch
time_limit_seconds = int(3600 * 1)
num_iters = 999_000_000
extra_name = ''
overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 1000 --time_limit_seconds {time_limit_seconds} --use_wandb true'

# ----------------------------------------------------

print(f'Games: {len(sats_games)}; Grid: {len(pgrid)}')
for params in pgrid:
    game_name = params['game']
    (alg_name, alg_config) = params['alg_and_config']
    dispatch.dispatch_experiments(alg_config, base_job_name=BASE_JOB_NAME, game_name=game_name, overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg=alg_name)

In [ ]:
# feb6_v2: re-run of feb6_v1, with 5h time limit and evals every 2h
sats_games = get_sats_games_from_prefixes(['feb4_4t'])
pgrid = ParameterGrid([{
    'game': [g for g in sats_games][:],
    'alg_and_config': [('ppo', 'ppo_jun8_23')]
}])
BASE_JOB_NAME = 'feb6_v2'
N_SEEDS = 11 # Number of seeds of each experiment to launch
time_limit_seconds = int(3600 * 5)
num_iters = 999_000_000
extra_name = ''
overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 7200 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 1000 --time_limit_seconds {time_limit_seconds} --use_wandb true'

# ----------------------------------------------------

print(f'Games: {len(sats_games)}; Grid: {len(pgrid)}')
for params in pgrid:
    game_name = params['game']
    (alg_name, alg_config) = params['alg_and_config']
    dispatch.dispatch_experiments(alg_config, base_job_name=BASE_JOB_NAME, game_name=game_name, overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg=alg_name)

In [39]:
# feb6_v1: feb4 4-type game; 11 seeds of PPO (after bugfix for NashConv and evals)
sats_games = get_sats_games_from_prefixes(['feb4_4t'])
pgrid = ParameterGrid([{
    'game': [g for g in sats_games][:],
    'alg_and_config': [('ppo', 'ppo_jun8_23')]
}])
BASE_JOB_NAME = 'feb6_v1'
N_SEEDS = 11 # Number of seeds of each experiment to launch
time_limit_seconds = int(3600 * 1)
num_iters = 999_000_000
extra_name = ''
overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 1000 --time_limit_seconds {time_limit_seconds} --use_wandb true'

# ----------------------------------------------------

print(f'Games: {len(sats_games)}; Grid: {len(pgrid)}')
for params in pgrid:
    game_name = params['game']
    (alg_name, alg_config) = params['alg_and_config']
    dispatch.dispatch_experiments(alg_config, base_job_name=BASE_JOB_NAME, game_name=game_name, overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg=alg_name)

Games: 20; Grid: 20
Submitted batch job 1082117
Submitted batch job 1082118
Submitted batch job 1082119
Submitted batch job 1082120
Submitted batch job 1082121
Submitted batch job 1082122
Submitted batch job 1082123
Submitted batch job 1082124
Submitted batch job 1082125
Submitted batch job 1082126
Submitted batch job 1082127
Submitted batch job 1082128
Submitted batch job 1082129
Submitted batch job 1082130
Submitted batch job 1082131
Submitted batch job 1082132
Submitted batch job 1082133
Submitted batch job 1082134
Submitted batch job 1082135
Submitted batch job 1082136
Submitted batch job 1082137
Submitted batch job 1082138
Submitted batch job 1082139
Submitted batch job 1082140
Submitted batch job 1082141
Submitted batch job 1082142
Submitted batch job 1082143
Submitted batch job 1082144
Submitted batch job 1082145
Submitted batch job 1082146
Submitted batch job 1082147
Submitted batch job 1082148
Submitted batch job 1082149
Submitted batch job 1082150
Submitted batch job 1082151


In [38]:
# feb4 4-type game; 11 seeds of CFR/CFR-no-tremble/PPO
sats_games = get_sats_games_from_prefixes(['feb4_4t'])
pgrid = ParameterGrid([{
    'game': [g for g in sats_games][:],
    'alg_and_config': [('cfr', 'cfr_port_10_ext'), ('ppo', 'ppo_jun8_23')]
}])
BASE_JOB_NAME = 'feb5_v2'
N_SEEDS = 11 # Number of seeds of each experiment to launch
time_limit_seconds = int(3600 * 1)
num_iters = 999_000_000
extra_name = ''
overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 1000 --time_limit_seconds {time_limit_seconds} --use_wandb true'

# ----------------------------------------------------

print(f'Games: {len(sats_games)}; Grid: {len(pgrid)}')
for params in pgrid:
    game_name = params['game']
    (alg_name, alg_config) = params['alg_and_config']
    dispatch.dispatch_experiments(alg_config, base_job_name=BASE_JOB_NAME, game_name=game_name, overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg=alg_name)

Games: 20; Grid: 40
Submitted batch job 1081386
Submitted batch job 1081387
Submitted batch job 1081388
Submitted batch job 1081389
Submitted batch job 1081390
Submitted batch job 1081391
Submitted batch job 1081392
Submitted batch job 1081393
Submitted batch job 1081394
Submitted batch job 1081395
Submitted batch job 1081396
Submitted batch job 1081397
Submitted batch job 1081398
Submitted batch job 1081399
Submitted batch job 1081400
Submitted batch job 1081401
Submitted batch job 1081402
Submitted batch job 1081403
Submitted batch job 1081404
Submitted batch job 1081405
Submitted batch job 1081406
Submitted batch job 1081407
Submitted batch job 1081408
Submitted batch job 1081409
Submitted batch job 1081410
Submitted batch job 1081411
Submitted batch job 1081412
Submitted batch job 1081413
Submitted batch job 1081414
Submitted batch job 1081415
Submitted batch job 1081416
Submitted batch job 1081417
Submitted batch job 1081418
Submitted batch job 1081419
Submitted batch job 1081420


In [19]:
# other example jobs below

In [6]:
# ### Just GOF CFR
# num_iters = 999_000_000
# time_limit_seconds = int(3600 * 1)
# for config in pgrid:
#     CONFIG = 'cfr' # The YML network config folder for the main jobs
#     N_SEEDS = 3 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 1800 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 50 --time_limit_seconds {time_limit_seconds} --use_wandb true'      
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jan18_cfr_v6_seeds_v3', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr', start_seed=200)

In [10]:
num_iters = 999_000_000
time_limit_seconds = int(3600 * 1)
for config in pgrid:
    CONFIG = 'cfr_10' # The YML network config folder for the main jobs
    N_SEEDS = 5 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 250 --time_limit_seconds {time_limit_seconds} --use_wandb true'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='feb4_v1', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

Submitted batch job 1079913


In [10]:
num_iters = 999_000_000
time_limit_seconds = int(3600 * 1 / 60)
for config in pgrid:
    CONFIG = 'ppo_jun8_23' # The YML network config folder for the main jobs
    N_SEEDS = 5 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 250 --time_limit_seconds {time_limit_seconds} --use_wandb true'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='feb3_test_short_v4', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='ppo')

Submitted batch job 1080222
Submitted batch job 1080223
Submitted batch job 1080224
Submitted batch job 1080225
Submitted batch job 1080226
Submitted batch job 1080227
Submitted batch job 1080228
Submitted batch job 1080229
Submitted batch job 1080230
Submitted batch job 1080231
Submitted batch job 1080232
Submitted batch job 1080233
Submitted batch job 1080234
Submitted batch job 1080235
Submitted batch job 1080236
Submitted batch job 1080237
Submitted batch job 1080238
Submitted batch job 1080239
Submitted batch job 1080240
Submitted batch job 1080241
Submitted batch job 1080242
Submitted batch job 1080243
Submitted batch job 1080244
Submitted batch job 1080245
Submitted batch job 1080246
Submitted batch job 1080247
Submitted batch job 1080248
Submitted batch job 1080249
Submitted batch job 1080250
Submitted batch job 1080251
Submitted batch job 1080252
Submitted batch job 1080253
Submitted batch job 1080254
Submitted batch job 1080255
Submitted batch job 1080256
Submitted batch job 

In [37]:
!squeue -u $USER --format="%.18i %.90j %.2t .%.10M%.6D %R"

             JOBID                                                                                       NAME ST .      TIME NODES NODELIST(REASON)
           1080121                                                                                    jupyter  R .2-07:39:51     1 ada21


In [36]:
!scancel 1081263 1081264 1081265 1081266 1081267 1081268 1081269 1081270 1081271 1081272 1081273 1081274 1081275 1081276 1081277 1081278 1081279 1081280 1081281 1081282 1081283 1081284 1081285 1081286 1081287 1081288 1081289 1081290 1081291 1081292 1081293 1081294 1081295 1081296 1081297 1081298 1081299 1081300 1081301 1081302 1081303 1081304 1081305 1081306 1081307 1081308 1081309 1081310 1081311 1081312 1081313 1081314 1081315 1081316 1081317 1081318 1081319 1081320 1081321 1081322 1081323 1081324 1081325 1081326 1081327 1081328 1081329 1081330 1081331 1081332 1081333 1081334 1081335 1081336 1081337 1081338 1081339 1081340 1081341 1081342 1081343 1081344 1081345 1081346 1081347 1081348 1081349 1081350 1081351 1081352 1081353 1081354 1081355 1081356 1081357 1081358 1081359 1081360 1081361 1081362 1081363 1081364 1081365 1081366 1081367 1081368 1081369 1081370 1081371 1081372 1081373 1081374 1081375 1081376 1081377 1081378 1081379 1081380 1081381 1081382 1081383 1081384 1081385 1081260 1081261 1081262 1081252 1081253 1081254 1081255 1081256 1081257 1081258 1081259 1081243 1081244 1081245 1081246 1081247 1081248 1081249 1081250 1081251 1081234 1081235 1081236 1081237 1081238 1081239 1081240 1081241 1081242 1081226 1081227 1081228 1081229 1081230 1081231 1081232 1081233 1081224 1081225 1081223


In [35]:
!squeue -u $USER --format="%.18i %.65j %.2t %.10M %.6D %R" | grep -v "jupyter" | awk '{print $1}'

JOBID
1081263
1081264
1081265
1081266
1081267
1081268
1081269
1081270
1081271
1081272
1081273
1081274
1081275
1081276
1081277
1081278
1081279
1081280
1081281
1081282
1081283
1081284
1081285
1081286
1081287
1081288
1081289
1081290
1081291
1081292
1081293
1081294
1081295
1081296
1081297
1081298
1081299
1081300
1081301
1081302
1081303
1081304
1081305
1081306
1081307
1081308
1081309
1081310
1081311
1081312
1081313
1081314
1081315
1081316
1081317
1081318
1081319
1081320
1081321
1081322
1081323
1081324
1081325
1081326
1081327
1081328
1081329
1081330
1081331
1081332
1081333
1081334
1081335
1081336
1081337
1081338
1081339
1081340
1081341
1081342
1081343
1081344
1081345
1081346
1081347
1081348
1081349
1081350
1081351
1081352
1081353
1081354
1081355
1081356
1081357
1081358
1081359
1081360
1081361
1081362
1081363
1081364
1081365
1081366
1081367
1081368
1081369
1081370
1081371
1081372
1081373
1081374
1081375
1081376
1081377
1081378
1081379
1081380
1081381
1081382
1081383
1081384
1081385
1081260
10

In [9]:
!squeue -u $USER | wc -l

102


In [17]:
def check_queue():
    !ssh borg.cs.ubc.ca '/opt/slurm/bin/squeue -u $USER --format="%.18i %.90j %.2t .%.10M %.6D %R"'


In [18]:
check_queue()

             JOBID                                                                                       NAME ST .      TIME  NODES NODELIST(REASON)
           1079910                                                                                    jupyter  R .     49:46      1 ada21
           1079915 jan12_repro_4t_jan12_repro_4t_1_base_dev1000_rho1_t4-ppo_jun8_23ppo_76-1350985_who_cares_p  R .      1:27      1 ada01
           1079913 jan12_repro_4t_jan12_repro_4t_1_base_dev1000_rho1_t4-cfr_10external_plus_linear-100_who_ca  R .     36:32      1 ada01


In [ ]:
!squeue | grep "may30" | wc -l

In [ ]:
!scancel -u ubuntu

In [20]:
!scancel  1079920 

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [ ]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

In [ ]:
Evaluation.objects.last()

In [ ]:
os.system("ssh borg.cs.ubc.ca '/opt/slurm/bin/sbatch'")

In [ ]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "eval" | awk '{print $1}' | xargs scancel

In [ ]:
from compress_pickle import dumps, loads


In [ ]:
loads?

In [ ]:
from compress_pickle.compressers.registry import get_known_compressions

In [ ]:
get_known_compressions()

In [ ]:
x = EquilibriumSolverRunCheckpoint.objects.last().policy

In [ ]:
len(x)

In [ ]:
len(dumps(pickle.loads(x), compression='bz2'))

In [ ]:
len(pickle.dumps(pickle.loads(x)))

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().walltime

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().t

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().bestresponse_set.all()

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().policy)

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().bestresponse_set.all().first().model)

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().bestresponse_set.all().last().model)